In [41]:
import os
current_directory = os.getcwd()
folder_name = os.path.basename(current_directory)
number = folder_name

In [42]:
import platform
platform_name = platform.system()

In [43]:
CONFIG = {
    'test_path': 'test.csv',
    'train_path': 'train.csv',

    'DEVICE' : 'cuda',
    'SEED' : 42,
}
if platform_name == 'Windows':
    CONFIG['data_main'] = 'C:/Users/Николай/PycharmProjects/CIBMTR/D.Data/main/'
    CONFIG['model_path'] = f"{number}_model.pkl"
else:
    CONFIG['data_main'] = '/kaggle/input/equity-post-HCT-survival-predictions/'
    CONFIG['model_path'] = "/kaggle/input/2.1.7_model.pkl/other/default/1/2.1.7_model.pkl"

In [44]:

import pandas as pd
import joblib 

In [45]:
train = pd.read_csv(f"{CONFIG['data_main']}{CONFIG['train_path']}")

num_columns = [col for col in train.columns if pd.api.types.is_numeric_dtype(train[col])]
cat_columns = [col for col in train.columns if col not in num_columns]

In [46]:
test = pd.read_csv(f"{CONFIG['data_main']}test.csv")

for col in test.columns: 
    if col in cat_columns:
        test[col] = test[col].astype(str)
        test[col] = test[col].fillna('-1')
    elif col in num_columns:
        test[col] = test[col].fillna(test[col].mode()[0])
        
for col in train.columns: 
    if col in cat_columns:
        train[col] = train[col].astype(str)
        train[col] = train[col].fillna('-1')
    elif col in num_columns:
        train[col] = train[col].fillna(train[col].mode()[0])
train = pd.get_dummies(train, columns=cat_columns)
test = pd.get_dummies(test, columns=cat_columns)

In [47]:
missing_cols = set(train.columns) - set(test.columns)

# Добавляем недостающие колонки в test и заполняем их значениями -1
for col in missing_cols:
    test[col] = '-1'

C:\Users\Николай\AppData\Local\Temp\ipykernel_13064\2018409134.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[col] = '-1'
C:\Users\Николай\AppData\Local\Temp\ipykernel_13064\2018409134.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[col] = '-1'
C:\Users\Николай\AppData\Local\Temp\ipykernel_13064\2018409134.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

In [48]:
final_cat = joblib.load(CONFIG['model_path'])

In [49]:
final_cat.get_params()

{'iterations': 966,
 'learning_rate': 0.008066583652537124,
 'depth': 8,
 'l2_leaf_reg': 5.142349241791732,
 'model_size_reg': 0,
 'rsm': 1,
 'loss_function': 'RMSE',
 'border_count': 711,
 'feature_border_type': 'GreedyLogSum',
 'nan_mode': 'Forbidden',
 'leaf_estimation_iterations': 1,
 'leaf_estimation_method': 'Newton',
 'random_seed': 42,
 'use_best_model': False,
 'logging_level': 'Silent',
 'max_ctr_complexity': 15,
 'one_hot_max_size': 50,
 'random_strength': 0,
 'eval_metric': 'RMSE',
 'bagging_temperature': 0.611469403382424,
 'allow_writing_files': False,
 'task_type': 'GPU',
 'bootstrap_type': 'Bayesian',
 'early_stopping_rounds': 250,
 'grow_policy': 'Lossguide',
 'max_leaves': 33,
 'score_function': 'NewtonL2',
 'leaf_estimation_backtracking': 'AnyImprovement',
 'boost_from_average': True}

In [50]:
preds_test = final_cat.predict(test.drop(columns=['ID']))

In [51]:
y_pred = test[['ID']].copy(deep=True)
y_pred["prediction"] = preds_test
y_pred.to_csv('submission.csv', index=False)

In [52]:
y_pred

,ID,prediction
0,28800,0.529230
1,28801,0.663982
2,28802,0.504143
